# Create a local information system of a Thin-Section

---

Table of Content

1. [Import files](#1-import-files)
2. [Create Raster tiles using Gdal](#2-create-raster-tiles-using-gdal)
3. [Convert SHP to Geojson using GDAL ogr2ogr](#3-convert-shp-to-geojson-using-gdal-ogr2ogr)
4. [Add GeoJson overlay to web-viewer](#4-add-geojson-overlay-to-web-viewer)

---

requirements:

- Python 3
- GDAL

In this section we collect input data and global settings.
We assume that we have a raster file of a Thin-section digitalized in any way and we have defined a polygon feature that overlay minerals or other elements of the thin-section.

The poligon feature layer must be in a GIS-readable format.
We use gadl2tiles with no-crs definition

In [1]:
# verify Python version
import sys
if sys.version_info[0] < 3:
    raise Exception("Python 3 not detected.")
else :
    print("Python 3 detected. OK.")

# verify GDAL version
try:
    from osgeo import gdal
    print(f"GDAL version: {gdal.__version__}")
except ImportError:
    raise ImportError("GDAL not detected.")

Python 3 detected. OK.
GDAL version: 3.8.5


# 1. Import files

In this section we use `upload_files_widgets` from `lis_function.py` to import data and files usinng widgets.

Next, store the file in a local subfolder with the same name of "project name" gived by the user.

## 1.1 File specification details

1. <u>**Shapefile**</u>: the minimum collection of 4 files [.shp, .dbf, .shx, .cpg] in ESRI standard format. the shapefile will be converted in GeoJSON format by `ogr2ogr` subprocess. The Shapefile must cointain polygons of mineral of the thin section. The element that overlay the raster image.

**Field table definition used in this LIS and stored in Shapefile**
|Field name|type|example value|NOTE|
|---|---|---|---|
|*Mineral*|str|`Pl`|Coded name of mineral|
|O|float|`90.0`|Degree of orientation|
|Asr|float|`0.39438`|Aspect Ratio|
|A|float|`0.12928`|Area in micrometers|
|R|float|`0.58411`|Roundness|
|GSI|float|`1.87833`|Grain Shape Index|

The code of minerals to use is reported in the following tablex:
|*Mineral*|Code|
|---|---|
|Amph|Amphibole|
|Ep|Epidote|
|Ap|Apatite|
|Kfs|K-Feldspar|
|Ol|Olivine|
|Pl|Plagioclase|
|Px|Pyroxene|
|Qtz|Quartz|
<!-- |Cal|Calcite|
|Ca-Si min|Calc-Silicate Mineral|
|Scap (aggr)|Scapolite (aggregate)|
|Scap|Scapolite|
|Oth|Other| -->


In this example polygons are generated by these tools:
- [Micro-Fabric Analyzer (MFA)](https://www.mdpi.com/2220-9964/10/2/51) (Visalli et al, 2021).
- [Quantitative X-ray Map Analyser (Q-XRMA)](https://www.sciencedirect.com/science/article/pii/S0098300417306982) (Ortolano et al, 2018)



2. <u>**Image**</u>: The original file raster used for webview, will be created hundreds of raster-tiles, there are in a format readable by `gdal` and browser, we suggest Jpeg or TIFF format

3. <u>**NO-Coordinate Reference System (NO-CRS)**</u>: check that both Shapefile and Image are correctly overlapped in a NO-CRS system. We only use the internal coordinate of the pixel of the raster, polygon vertex defined in Shapefile are referring to a specific pixel of the Imagefile.


In [2]:
# Input widgets

from lis_functions import upload_files_widgets
Shapefile_selector, Jpgfile_selector, projectname = upload_files_widgets()
# Shapefile_selector, Jpgfile_selector, projectname, EPSGcode = upload_files_widgets()


Label(value='Select ALL the shapefile files [.shp, .dbf, .shx, .cpg]')

FileUpload(value=(), accept='.shp, .dbf, .shx, .cpg', description='Shapefile polygon layer', multiple=True)

Label(value='Select the Image file')

FileUpload(value=(), description='Image')

Label(value='Select the project name')

Text(value='', description='Name:', placeholder='NAME')

Label(value='Note: ouptut folder will be created in the same directory as the input files')

In [3]:
from lis_functions import save_to_temp_dir

# Save file to a temporary directory

files_path = save_to_temp_dir(Shapefile_selector, Jpgfile_selector, projectname.value)

# Run gdal2tiles


# 2. Create Raster tiles using Gdal

Using a subprocess, we run the function `run_gdal2tiles`, that launch the gdal2tiles procedure in terminal.

By default a simple webviewer is written and stored in openalayer.html file inside the project folder.

NOTE: destination folder is also created by `save_to_temp_dir` function before

In [4]:
from lis_functions import run_gdal2tiles

# Create raster tiles using gdal2tiles
# for customization see function in lis_functions.py

run_gdal2tiles(files_path['Raster_path'], projectname.value)

# some information is stored in tilemapsoruce.xml file generated from gdal2tiles

WARNING! - Directory already exists, Files with same name will be overwrited.


Generating Base Tiles:


0...10...20...30...40...50...60...70...80...90...100 - done.
0..

Generating Overview Tiles:


.10...20...30...40...50...60...70...80...90...100 - done.
Tiles stored in: PAL22


# 3. Convert SHP to Geojson using GDAL ogr2ogr

Using `ogr2ogr` in a subprocess we convert shp in geoJson format


In [5]:
import os

from lis_functions import convert_shp_to_geojson

# Get the input shapefile path from `save_to_temp_dir` cell
input_shp = files_path['Shp_file_path']

# Create the output GeoJSON file path
output_geojson = projectname.value + '/' + os.path.basename(input_shp) +'.geojson'

# Convert SHP to GeoJSON using the function and an ogr2ogr subprocess
convert_shp_to_geojson(input_shp, output_geojson)

Conversion completed: PAL22/PAL22_min_porphy.shp.geojson


# 4. Add GeoJson overlay to web-viewer

We access the openalyers.html defautl file generated by gdal2tiles and add some simple javascript code to overlay the previously created GeoJSON.

1. We use the file `geojson_template.js`, a JavaScript file prepared with the script code to add to the mapviewer; in the next step we simply change the placeholder to the current GeoJSON file.
2. We create the javascript code and add it as a source script to the `openlayers.html` file generated earlier by the gdal2tiles subprocess

The visualization style for polygon overlay can be edited in javascript using openalyers syntax before this process.

**NOTE**: For a properly map overlay viweing in a browser, we assume a nominal reference system as EPSG:3857, the same as  used by OpenLayers by default. If other SRSs are used, the vector overlay on raster tiles is not correct.

This is a fictitious hypothesis only for the execution of the webviewer in HTML code. Like an hack that exploits the OpenLayer libraries, which are instead designed for maps with well-defined reference systems. It only works if the vector layer and raster layer are already overlaid (according to the numerical values of their respective coordinates), and have been saved without SRS definition.

In [2]:
from lis_functions import add_geojson_overlay_to_gdal2tiles_html_output

# Note: the default output of gdal2tiles is an html file named 'openlayers.html'
# in the output directory. This file is a template and can be modified to include

#html_input = 'PAL22/openlayers.html'
html_input = projectname.value + '/openlayers.html'
#html_output = 'PAL22/index.html'
html_output = projectname.value + '/index.html'
#geojson_file_path = 'PAL22/PAL22_min_porphy.shp.geojson'
geojson_file_path = output_geojson

add_geojson_overlay_to_gdal2tiles_html_output(geojson_file_path, html_input, html_output)


File updated: PAL22/index.html


# 5. Add popup to the map

We now embed the popular library "OpenLayer Extension" AKA `ol-ext` see [source code](https://github.com/Viglino/ol-ext?tab=readme-ov-file) released under BSD 3-Clause License. Copyright (c) 2016-2021, Jean-Marc Viglino, IGN-France [All rights reserved](https://github.com/Viglino/ol-ext?tab=BSD-3-Clause-2-ov-file) in html file.

the function called `add_popup_feature_to_gdal2tiles_html_output(Html_input,Html_output)` work on html file.
Steps are:
1. Add link to JS and CSS of `ol-ext` hosted by cdn.jsdelivr.net in the head and bottom of html
2. Add a precise customized script with `ol.Overlay.PopupFeature()`
3. Add customized CSS overrides for better visualization.

The customized popup work with precise field name in the geojson file used, in this example we have a template like this:


```javascript
template: {
        title: 
          function(f) {
            return f.get('Mineral'); // return the Mineral code
          },
        attributes: //
            {
            'O' : { 
                    title: 'Orientation',
                    before:'', 
                    format: ol.Overlay.PopupFeature.localString(), // format as local string APPROX
                    after:'°' 
                },
            'AsR' : { title: 'Aspect Ratio' },
            'R': { title: 'Roundness' },
            'A': { title: 'Area' },
            'GSI': {title: 'Grain Shape Index'} // ,
            }
        }
```


Change this in the function definition if are different on your field table.

In [3]:

from lis_functions import add_popup_feature_to_gdal2tiles_html_output

add_popup_feature_to_gdal2tiles_html_output('PAL22/index.html', 'PAL22/index_2.html')

File updated: PAL22/index_2.html


# 6. Add mineral legend and rosediagrams

Now we add to html another customization.
With *ArcStereoNet* [*] or other software for analizing microstructural data of Thin section, we obtain weighted and unweighted rose diagram based on grains cumulative area.

[*] Ortolano, Gaetano, Alberto D’Agostino, Mario Pagano, Roberto Visalli, Michele Zucali, Eugenio Fazio, Ian Alsop, and Rosolino Cirrincione. 2021. "ArcStereoNet: A New ArcGIS® Toolbox for Projection and Analysis of Meso- and Micro-Structural Data" ISPRS International Journal of Geo-Information 10, no. 2: 50. https://doi.org/10.3390/ijgi10020050

In the folder `templates\asset\` are stored SVGs files of rose diagrams and customized legend SVG (using the same color)

In the HTML now we ad at the bottom:
- a legend icon buttons for each mineral detected inside the Thin Section considered
- a blank rose diagram that be updated after the event of click on a specific mineral grain.
- a JS code for the features


In [15]:
# defining the function
from bs4 import BeautifulSoup
import os

# create a dictionary of legend icons paths
legend_icons = {
    'Ap': 'templates/asset/legend/Ap_legend.svg',
    'Amph': 'templates/asset/legend/Amph_legend.svg',
    'Ep': 'templates/asset/legend/Ep_legend.svg',
    'Kfs': 'templates/asset/legend/Kfs_legend.svg',
    'Ol': 'templates/asset/legend/Ol_legend.svg',
    'Pl': 'templates/asset/legend/Pl_legend.svg',
    'Px': 'templates/asset/legend/Px_legend.svg',
    'Qtz': 'templates/asset/legend/Qtz_legend.svg'
}

# set the path of rose diagrams images
# Create a dictionary to store the paths of the images

rose_diagrams = {
    'Amph_UnW': 'templates/asset/rose/PAL22_Amph_1.svg',
    'Amph_W': 'templates/asset/rose/PAL22_Amph_2.svg',
    'Ep_UnW': 'templates/asset/rose/PAL22_Ep_1.svg',
    'Ep_W': 'templates/asset/rose/PAL22_Ep_2.svg',
    'Kfs_UnW' : 'templates/asset/rose/PAL22_Kfs_1.svg',
    'Kfs_W' : 'templates/asset/rose/PAL22_Kfs_2.svg',
    'Ol_UnW' : 'templates/asset/rose/PAL22_Ol_1.svg',
    'Ol_W' : 'templates/asset/rose/PAL22_Ol_2.svg',
    'Oth_UnW' : 'templates/asset/rose/PAL22_Oth_1.svg',
    'Oth_W' : 'templates/asset/rose/PAL22_Oth_2.svg',
    'Pl_UnW' : 'templates/asset/rose/PAL22_Pl_1.svg',
    'Pl_W' : 'templates/asset/rose/PAL22_Pl_2.svg',
    'Px_UnW' : 'templates/asset/rose/PAL22_Px_1.svg',
    'Px_W' : 'templates/asset/rose/PAL22_Px_2.svg',
    'Qtz_UnW' : 'templates/asset/rose/PAL22_Qtz_1.svg',
    'Qtz_W' : 'templates/asset/rose/PAL22_Qtz_2.svg',
}

blankDiagram = 'templates/asset/rose/blank_diagram.svg'

#set the width of the map view after adding the legend icons
map_view_height='255px'

def add_legend_icons(Html_input,Html_output, legend_icons, map_view_height, blankdiagram):
    # open file
    with open(Html_input, 'r') as f:
        html_content = f.read()
    # parse with BeautifulSoup

    soup = BeautifulSoup(html_content, 'html.parser')

    # copy the files stored in legend_icons dictionary to the output subdirectory
    subdirectoryLEGEND = os.path.join(os.path.dirname(Html_output), 'templates','asset', 'legend')
    os.makedirs(subdirectoryLEGEND, exist_ok=True)

    for key in legend_icons:
        icon_path = legend_icons[key]
        icon_name = os.path.basename(icon_path)
        icon_output_path = os.path.join(subdirectoryLEGEND, icon_name)
        with open(icon_path, 'rb') as f:
            with open(icon_output_path, 'wb') as f1:
                f1.write(f.read())
    
    # add HTML tags for Legend icon
    new_div_tag = soup.new_tag('div')
    new_div_tag['id'] = 'legendbar'
    new_div_tag['style'] = 'display: flex; justify-content: center;'

    table_tag = soup.new_tag('table')
    tr_tag = soup.new_tag('tr')
    for key in legend_icons:
        td_tag = soup.new_tag('td')
        img_tag = soup.new_tag('img')
        img_tag['src'] = legend_icons[key]
        img_tag['id'] = f'{key}_legend'
        img_tag['onclick'] = f'legendClick(\'{key}\')' # call the Javascript legendClic(mineral) function with the mineral name
        td_tag.append(img_tag)
        tr_tag.append(td_tag)
    table_tag.append(tr_tag)
    new_div_tag.append(table_tag)
    # add the new div tag to the body
    soup.body.append(new_div_tag)
    
    #reduce the height of the map to make space for the legend
    style_tag = soup.find('style')
    if style_tag:
        css_content = style_tag.string
        # Modifica il CSS specifico
        css_content = css_content.replace('#map { height: 90%; border: 1px solid #888; }', f'#map {{ height: {map_view_height}; border: 1px solid #888; }}')
        style_tag.string = css_content

    # add CSS style for legend icons
    new_style_tag = soup.new_tag('style')
    css_string = "#ShowAll, #HideAll"
    for key in legend_icons:
        css_string += f", #{key}_legend"
    css_string += " {width: 80px; filter: grayscale(0);}"
    new_style_tag.string = css_string
    #new_style_tag.string = """#ShowAll, #Amph_legend, #Ep_legend, #Ap_legend, #Kfs_legend, #HideAll, #Ol_legend, #Pl_legend, #Px_legend, #Qtz_legend {width: 80px; filter: grayscale(0);}"""
    soup.head.append(new_style_tag)

    # ROSE DIAGRAMS file and paths

    # create the path for the rose diagrams
    # TESTING_zone/LIS/templates/asset/rose/PAL12_A_Amph_2.svg
    subdirectoryRoseD = os.path.join(os.path.dirname(Html_output), 'templates','asset', 'rose')
    os.makedirs(subdirectoryRoseD, exist_ok=True)

    #copy files stored in rose_diagrams dictionary to the output subdirectory
    for key in rose_diagrams:
        rose_diagram_path = rose_diagrams[key]
        rose_diagram_name = os.path.basename(rose_diagram_path)
        rose_diagram_output_path = os.path.join(subdirectoryRoseD, rose_diagram_name)
        with open(rose_diagram_path, 'rb') as f:
            with open(rose_diagram_output_path, 'wb') as f1:
                f1.write(f.read())
    
    # # copy the files stored in legend_icons dictionary to the output subdirectory
    # subdirectoryLEGEND = os.path.join(os.path.dirname(Html_output), 'templates','asset', 'legend')
    # os.makedirs(subdirectoryLEGEND, exist_ok=True)

    # for key in legend_icons:
    #     icon_path = legend_icons[key]
    #     icon_name = os.path.basename(icon_path)
    #     icon_output_path = os.path.join(subdirectoryLEGEND, icon_name)
    #     with open(icon_path, 'rb') as f:
    #         with open(icon_output_path, 'wb') as f1:
    #             f1.write(f.read())

    # write blank diagram svg in subfolder
    blankdiagram_name = os.path.basename(blankdiagram)
    blankdiagram_output_path = os.path.join(subdirectoryRoseD, blankdiagram_name)

    with open(blankdiagram, 'rb') as f:
        with open(blankdiagram_output_path, 'wb') as f1:
            f1.write(f.read())
    
    
    # add Javascript code


    JSscriptText ="""
    // Selection interaction
    const selectInteraction = new ol.interaction.Select({
        layers: [vectorLayer], //
        condition: ol.events.condition.click // Seleziona i poligoni al clic
    });
    map.addInteraction(selectInteraction);

    // listener for the selection interaction
    selectInteraction.on('select', function(event) {
        const selectedFeatures = event.selected;
        selectedFeatures.forEach(function(feature) {
            console.log('Poligono selezionato:', feature.getProperties());
            // ...
        });
    });

    function updateSVG(mineral) {
        SvgURI1	= $$$SVGURI1$$$;
        SvgURI2 = $$$SVGURI2$$$;

        var imgElement1 = document.getElementById('svg1');
        if (imgElement1) {
            imgElement1.src = SvgURI1;
            imgElement1.onerror = function() {
                imgElement1.src = 'blank_diagram.svg';
            }
        }
        var imgElement2 = document.getElementById('svg2');
        if (imgElement2) {
            imgElement2.src = SvgURI2;
            imgElement2.onerror = function() {
                imgElement2.src = 'blank_diagram.svg';
            }
        }
    }
    // Function to get full name of mineral from acronyms
    function getMineralName(mineral) {
        switch (mineral) {
            case 'Amph': return 'Amphibole';
            case 'Ep': return 'Epidote';
            case 'Ap': return 'Apatite';
            case 'Kfs': return 'K-Feldspar';
            case 'Ol': return 'Olivine';
            case 'Pl': return 'Plagioclase';
            case 'Px': return 'Pyroxene';
            case 'Qtz': return 'Quartz';
            case 'Cal': return 'Calcite';
            case 'Ca-Si min': return 'Calc-Silicate Mineral';
            case 'Scap (aggr)': return 'Scapolite (aggregate)';
            case 'Scap': return 'Scapolite';
            case 'Oth': return 'Other';
        }
    }

    function legendClick(mineral) {
    // Example: 'Qtz': 'templates/asset/legend/Qtz_legend.svg'
    // 'Qtz_UnW': 'templates/asset/rose_diagrams/PAL12_A_Qtz1.svg',
    // 'Qtz_W': 'templates/asset/rose_diagrams/PAL12_A_Qtz2.svg'
    // idLegend = mineral + '_legend';

    //addFilter(mineral);
    
    updateSVG(mineral);
    }
    """
    # replace the placeholders 
    #     'Qtz_W' : 'templates/asset/rose/PAL22_Qtz_2.svg',
    # TODO Fix the path of the rose diagrams
    JSscriptText = JSscriptText.replace("$$$SVGURI1$$$", f"'{os.path.dirname(blankdiagram)}' + mineral + '1.svg'")
    JSscriptText = JSscriptText.replace("$$$SVGURI2$$$", f"'{os.path.dirname(blankdiagram)}' + mineral + '2.svg'")

    new_script_tag = soup.new_tag('script')
    new_script_tag.string = JSscriptText
    soup.body.append(new_script_tag)


    # Add the rose diagrams tags with BeautifulSoup
    
    
   

    #adding HTML tags

    tableRoseDiagram = soup.new_tag('table')
    rowRoseDiagram = soup.new_tag('tr')
    columnRoseDiagram1 = soup.new_tag('td')
    RoseDiagram1 = soup.new_tag('img')
    RoseDiagram1['id'] = 'svg1'
    RoseDiagram1['src'] = blankdiagram
    RoseDiagram1['width'] = '100%'
    columnRoseDiagram1.append(RoseDiagram1)
    rowRoseDiagram.append(columnRoseDiagram1)
    columnRoseDiagram2 = soup.new_tag('td')
    RoseDiagram2 = soup.new_tag('img')
    RoseDiagram2['id'] = 'svg2'
    RoseDiagram2['src'] = blankdiagram
    RoseDiagram2['width'] = '100%'
    columnRoseDiagram2.append(RoseDiagram2)
    rowRoseDiagram.append(columnRoseDiagram2)
    tableRoseDiagram.append(rowRoseDiagram)
    soup.append(tableRoseDiagram)

        
    #write the modified html to a new file
    with open(Html_output, 'w') as file:
        file.write(soup.prettify())
    
    # copy the files stored in legend_icons dictionary to the output subdirectory
    subdirectory = os.path.join(os.path.dirname(Html_output), 'templates','asset', 'legend')
    os.makedirs(subdirectory, exist_ok=True)

    for key in legend_icons:
        icon_path = legend_icons[key]
        icon_name = os.path.basename(icon_path)
        icon_output_path = os.path.join(subdirectory, icon_name)
        with open(icon_path, 'rb') as f:
            with open(icon_output_path, 'wb') as f1:
                f1.write(f.read())
    
    

add_legend_icons('PAL22/index_2.html','PAL22/index_3.html', legend_icons, map_view_height, blankDiagram)